In [1]:
import os
os.chdir('..')

In [2]:
from team_functions import *

In [3]:
today = date.today()
print(f'Today is: {today}')

Today is: 2024-03-11


In [4]:
last_games = pd.read_excel('Predictions/games_odds.xlsx', 'Tonight')
prev_games = pd.read_excel('Predictions/games_odds.xlsx', 'Previous')

In [5]:
options = Options()
options.add_argument('--headless')
driver = Firefox(options=options)

In [6]:
url = 'https://www.basketball-reference.com/boxscores/'
soup = getSoup(url, driver=driver)

In [7]:
summaries = soup.find('div', {'class': 'game_summaries'})
games = summaries.findAll('table', {'class': 'teams'})

games_df = []
for idx, game in enumerate(games):
    loser = game.find('tr', {'class': 'loser'}).find('td').find('a').get('href').split('/')[2]
    winner = game.find('tr', {'class': 'winner'}).find('td').find('a').get('href').split('/')[2]
    
    game_row = {'Winner': winner, 'Loser': loser}
    games_df.append(game_row)
    
games_df = pd.DataFrame(games_df)

In [10]:
results = []
for team in last_games['visitor']:
    if team in list(games_df['Winner']):
        results.append(1)
    elif team in list(games_df['Loser']):
        results.append(0)
    else:
        print(f'Error checking results of team: {team}')

last_games['Results'] = np.array(results)

In [11]:
last_games

,date,visitor,local,v_line,l_line,Results
0,2024-03-09,CHI,LAC,3.200000,1.370370,0
1,2024-03-09,DAL,DET,1.327869,3.450000,1
2,2024-03-09,BRK,CHO,1.420168,2.950000,0
3,2024-03-09,BOS,PHO,1.458716,2.800000,1
4,2024-03-09,SAS,GSW,4.600000,1.212766,1
5,2024-03-09,UTA,DEN,6.500000,1.125000,0
6,2024-03-09,TOR,POR,1.704225,2.200000,0


In [12]:
prev_games.tail()

,date,visitor,local,v_line,l_line,Results
201,2024-03-08,MIN,CLE,1.952381,1.869565,0
202,2024-03-08,MIA,OKC,3.850000,1.277778,0
203,2024-03-08,ATL,MEM,1.704225,2.200000,1
204,2024-03-08,MIL,LAL,1.769231,2.100000,0
205,2024-03-08,HOU,POR,1.444444,2.850000,1


In [13]:
prev_games = pd.concat([prev_games, last_games])

In [14]:
url = 'https://sportsbook.draftkings.com/leagues/basketball/nba'

In [15]:
driver.get(url)

html = driver.page_source
soup = bs(html, 'lxml')

driver.close()

In [16]:
def american_to_decimal(american):
    '''
    This functions transform an american system odd (e.g. -120) to a decimal system (e.g 1.8)
    
    INPUTS:
    player: row containing player stats
    
    OUTPUT:
    new_player: copy of the player df input with the updated Odd value in Decimal System
    '''
    # Check if the value is positive or negative (e.g. Odd = +100 or Odd = -120)
    # Transform it to decimal with the given formula
    if american > 0:
        decimal = (american/100) + 1
    else:
        decimal = (100/abs(american)) + 1
    
    return decimal

In [17]:
tables = soup.findAll('tbody', {'class': 'sportsbook-table__body'})
rows = []
for table in tables:
    for row in table.findAll('tr'):
        rows.append(row)
games, game, idx = [], {'date': today, 'visitor': '', 'local': '', 'v_line': 0, 'l_line': 0}, 0
for row in rows:
    team_name = row.find('div', {'class': 'event-cell__name-text'}).text.split(' ')
    if len(team_name[0]) == 3:
        team_id = team_name[0]
    else:
        team_id = team_name[0] + team_name[1][0]
    team_id = team_id.replace('BKN', 'BRK').replace('CHA', 'CHO')
    odd = row.find('span', {'class': 'sportsbook-odds american no-margin default-color'}).text.replace('−', '-')
    
    if idx == 0:
        game['visitor'] = team_id
        game['v_line'] = american_to_decimal(int(odd))
        idx = 1
        
    elif idx == 1:
        game['local'] = team_id
        game['l_line'] = american_to_decimal(int(odd))
        idx = 0
        
        games.append(game)
        game = {'date': today, 'visitor': '', 'local': '', 'v_line': 0, 'l_line': 0}
    
games_df = pd.DataFrame(games)
games_df

,date,visitor,local,v_line,l_line
0,2024-03-11,CHO,DET,2.300000,1.645161
1,2024-03-11,PHO,CLE,1.420168,2.950000
2,2024-03-11,DAL,CHI,1.571429,2.450000
3,2024-03-11,GSW,SAS,1.540541,2.540000
4,2024-03-11,TOR,DEN,8.500000,1.083333
5,2024-03-11,BOS,POR,1.227273,4.400000


In [18]:
with pd.ExcelWriter('Predictions/games_odds.xlsx') as writer:  
    games_df.to_excel(writer, sheet_name='Tonight', index=0)
    prev_games.to_excel(writer, sheet_name='Previous', index=0)